In [28]:
# !pip install scikit-learn
# !pip install keras
# !pip install tensorflow
# !pip install pandas
from keras import Sequential
from keras.layers import Dense, LSTM, Activation, Dropout, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
# Load Data
df = pd.read_csv("dataset_phishing.csv")

In [3]:
df.head()

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [4]:
unique_statuses = df['status'].unique()

# Convert to a list if needed
unique_statuses_list = unique_statuses.tolist()

# Display the unique categories
print(unique_statuses_list)

['legitimate', 'phishing']


In [6]:
url_status = df[['url', 'status']]
labelencoder = LabelEncoder()
url_status['status_encoded'] = labelencoder.fit_transform(url_status['status'])
url_status_encoded = url_status[['url','status_encoded']]

/tmp/ipykernel_658278/4159172604.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  url_status['status_encoded'] = labelencoder.fit_transform(url_status['status'])


In [7]:
url_status_encoded.head()

,url,status_encoded
0,http://www.crestonwood.com/router.php,0
1,http://shadetreetechnology.com/V4/validation/a...,1
2,https://support-appleld.com.secureupdate.duila...,1
3,http://rgipt.ac.in,0
4,http://www.iracing.com/tracks/gateway-motorspo...,0


In [18]:
df['url_length'] = df['url'].str.len()

# Step 2: Find the maximum length
(df['url_length'].max())

1641

In [43]:
# Tokenization
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(url_status_encoded['url'])
sequences = tokenizer.texts_to_sequences(url_status_encoded['url'])

# Padding
max_len = 1650
X = pad_sequences(sequences, maxlen = max_len)

# 4. Splitting Data
X_train, X_test, y_train, y_test = train_test_split(X, url_status_encoded['status_encoded'], test_size=0.2, random_state=42, stratify=url_status_encoded['status_encoded'])

# 5. Model Architecture
vocab_size = len(tokenizer.word_index) + 1  # Total number of unique characters

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length = max_len))  # Embedding Layer
# model.add(LSTM(512, return_sequences=True,dropout=0.2))# First LSTM Layer
model.add(Dense(512,activation='relu'))
# model.add(LSTM(512, return_sequences=False,dropout=0.2))
model.add(Dense(1, activation='sigmoid'))  # Output Layer for binary classification

# 6. Compiling the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 7. Training the Model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

# 8. Evaluating the Model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))

Epoch 1/5


/common/home/projectgrps/IS424/IS424G2/jupyterlab-venv-tf-217/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 1650)

In [ ]:
# Sample dont use
'''
# 4. Splitting Data
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# 5. Model Architecture
vocab_size = len(tokenizer.word_index) + 1  # Total number of unique characters

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len))  # Embedding Layer
model.add(LSTM(128, return_sequences=True))  # First LSTM Layer
model.add(Dropout(0.2))
model.add(LSTM(64))  # Second LSTM Layer
model.add(Dense(1, activation='sigmoid'))  # Output Layer for binary classification

# 6. Compiling the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 7. Training the Model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

# 8. Evaluating the Model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))
'''